#### Question #1 

Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from env import username, password, host

import warnings
warnings.filterwarnings("ignore")


In [70]:
def acquire_zillow(use_cache=True):
    '''
    '''
    
    if os.path.exists('zillow.csv') and use_cache:
        print('Using cached csv')
        return pd.read_csv('zillow.csv')
    print('Acquiring data from SQL database')

    database_url_base = f'mysql+pymysql://{username}:{password}@{host}/zillow'
    query ='''
    SELECT bedroomcnt AS bedrooms, 
           bathroomcnt AS bathrooms, 
           calculatedfinishedsquarefeet AS finished_sqft, 
           taxvaluedollarcnt AS tax_value,
           yearbuilt AS yr_built,
           taxamount AS tax_amount,
           fips
    FROM properties_2017
    WHERE propertylandusetypeid = '261';
    '''
    df = pd.read_sql(query, database_url_base)
    df.to_csv('zillow.csv', index=False)
    return df

acquire_zillow()

Using cached csv


,bedrooms,bathrooms,finished_sqft,tax_value,yr_built,tax_amount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [71]:
df=acquire_zillow()

Using cached csv


In [72]:
# Lots of nulls ad NaNs!!!
df.isnull().sum()

bedrooms           11
bathrooms          11
finished_sqft    8484
tax_value         493
yr_built         9337
tax_amount       4442
fips                0
dtype: int64

#### Question #2 

Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [73]:
# Drop any nulls. Considering the size of our database, this
# large amount of drops does not reflect poorly on our population
# of over 2 million rows.
df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   bedrooms       float64
 1   bathrooms      float64
 2   finished_sqft  float64
 3   tax_value      float64
 4   yr_built       float64
 5   tax_amount     float64
 6   fips           float64
dtypes: float64(7)
memory usage: 130.6 MB


In [74]:
df.shape

(2140235, 7)

In [75]:
# Updated Nulls and NaN amounts
df.isnull().sum()

bedrooms         0
bathrooms        0
finished_sqft    0
tax_value        0
yr_built         0
tax_amount       0
fips             0
dtype: int64

In [76]:
# Change all float values to integers. Dropping the decimals off
# the decimals off of the tax values shouldn't negatively effect
# the outcome.
df = df.astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column         Dtype
---  ------         -----
 0   bedrooms       int64
 1   bathrooms      int64
 2   finished_sqft  int64
 3   tax_value      int64
 4   yr_built       int64
 5   tax_amount     int64
 6   fips           int64
dtypes: int64(7)
memory usage: 130.6 MB


In [77]:
df.head()

,bedrooms,bathrooms,finished_sqft,tax_value,yr_built,tax_amount,fips
4,4,2,3633,296425,2005,6941,6037
6,3,4,1620,847770,2011,10244,6037
7,3,2,2077,646760,1926,7924,6037
11,0,0,1200,5328,1972,91,6037
14,0,0,171,6920,1973,255,6037
